# Pandas Data pre-processing 판다스 데이터 전처리

1. Hierarchical Indexing 계층 색인 

2. Sorting 정렬

3. 

4. 

## 1. Hierarchical Indexing 계층 색인 
- 행, 열의 각 축에 대해 다중 단계(계층)를 지정하여 데이터에 차원을 설정
- 인덱스에 다차원 리스트를 전달하면 계층 색인을 지정할 수 있음
- 데이터 구조를 재배열하거나 pivot 테이블과 같은 그룹 기반 작업에 유용
- 재배열 메서드
    - stack() : 컬럼을 로우로 피벗
    - unstack() : 로우를 컬럼으로 피벗

In [1]:
import pandas as pd
import numpy as np

### 1.0. Series hierarchical indexing
- 인덱스에 다차원 리스트(아이템 2개)를 전달
- 다차원리스트[0] : 상위계층
- 상위계층 작성시 주의점 : 각 계층별로 속하는 하위계층 값의 개수만큼 계층명 작성
- 상위계층 리스트 개수 = 하위계층 리스트 개수
- 다차원리스트[1] : 하위계층

### 1.1. double level row Series
- row index
- 상위계층 : a, b, c, d
- 하위계층 : a(1, 2, 3), b(1, 2), c(1, 2, 3, 4), d(1)

In [2]:
sr_1 = pd.Series(np.arange(10),
               index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'c', 'd'],
                     [1, 2, 3, 1, 2, 1, 2, 3, 4, 1]])
sr_1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [3]:
# 인덱스 확인

sr_1.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('c', 1),
            ('c', 2),
            ('c', 3),
            ('c', 4),
            ('d', 1)],
           )

In [4]:
# 상위 idx(상위 계층)에 접근
# 계층 색인이 적용된 객체의 상위 index에 접근: 일반적인 Series indexing으로 접근

sr_1['c']

1    5
2    6
3    7
4    8
dtype: int32

- slicing
    - 계층색인에 대한 slicing: last index도 포함 > 'b':'d' 입력 > 'b', 'c', 'd' 조회

In [5]:
sr_1['b':'d']

b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [6]:
# 하위 계층에 접근
# index a, b, c, d에서 하위계층 label index=2 data 조회
# `sr[상위계층범위, 하위계층번호]`

sr_1[:, 2]

a    1
b    4
c    6
dtype: int32

In [7]:
# 하위 계층 slicing: loc method 사용
# 상위 계층 c > 하위계층 2~4(user custom index) slicing > last index 포함

sr_1['c'].loc[2:4]

2    6
3    7
4    8
dtype: int32

### 1.2. `sr.unstack()`

- 최하위(기본동작)에 있는 row계층을 columns로 적용하여 위계층으로 올림

- `Series` object > `DataFrame` object로 재배열

- `NaN`: 기존에는 없던 row 계층에 대한 값

In [8]:
# 현재 row만 2계층, columns 0계층

sr_1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [9]:
df_1 = sr_1.unstack()

df_1

,1,2,3,4
a,0.0,1.0,2.0,NaN
b,3.0,4.0,NaN,NaN
c,5.0,6.0,7.0,8.0
d,9.0,NaN,NaN,NaN


### 1.3. `df.stack()`

- columns에 할당된 값을 > multi index의 lower index로 내려서 재배열

- DataFrame data type > Series data type

In [10]:
stk = df_1.stack()

stk

a  1    0.0
   2    1.0
   3    2.0
b  1    3.0
   2    4.0
c  1    5.0
   2    6.0
   3    7.0
   4    8.0
d  1    9.0
dtype: float64

### 1.4. double level row * double level columns DataFrame

In [11]:
# DataFrame 생성
# 구조 : 4 x 5
# 로우 인덱스 : 상위(2017, 2018) / 하위(모든 상위 인덱스에 대해 동일하게 'a', 'b')
# 컬럼 인덱스 : 상위(서울, 경기) / 하위({서울:강남, 잠실} / 경기-분당, 수원, 판교)
# 값 : 1씩 증가하는 20개
my_df = pd.DataFrame(np.arange(20).reshape(4, 5),
                 index=[[2017, 2017, 2018, 2018],
                       ['a', 'b', 'a', 'b']],
                 columns=[['서울', '서울', '경기', '경기', '경기'],
                         ['강남', '잠실', '분당', '수원', '판교']])
my_df

서울      경기        
        강남  잠실  분당  수원  판교
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

In [12]:
# colums의 higher 계층 접근: 일반적인 df columns indexing

my_df['서울']

강남  잠실
2017 a   0   1
     b   5   6
2018 a  10  11
     b  15  16

In [13]:
# double hierarchical columns > double indexing

my_df['서울']['강남']

2017  a     0
      b     5
2018  a    10
      b    15
Name: 강남, dtype: int32

In [14]:
# 한 번에 두 계층 index 지정 > tuple로 전달

my_df[('서울', '강남')]

2017  a     0
      b     5
2018  a    10
      b    15
Name: (서울, 강남), dtype: int32

In [15]:
# row의 상위 계층 접근

my_df.loc[2017]

서울    경기      
  강남 잠실 분당 수원 판교
a  0  1  2  3  4
b  5  6  7  8  9

In [16]:
# row의 하위 계층 접근
# 1. double indexing

my_df.loc[2017].loc['a']

서울  강남    0
    잠실    1
경기  분당    2
    수원    3
    판교    4
Name: a, dtype: int32

In [17]:
# 2. tuple로 전달 > double list X

my_df.loc[(2017, 'a')]

서울  강남    0
    잠실    1
경기  분당    2
    수원    3
    판교    4
Name: (2017, a), dtype: int32

In [18]:
# 분당 ~ 수원까지 데이터 조회
# caution: DataFrame columns > row slicing result에만 slicing 적용 가능
# df[first level col][second level col slicing '':''] > Error: columns slicing 단독으로 불가능

my_df['경기'].loc[:, '분당':'수원']

분당  수원
2017 a   2   3
     b   7   8
2018 a  12  13
     b  17  18

In [19]:
# `my_df` row 최하위계층 > columns 재배열

my_df.unstack()

서울              경기                    
      강남      잠실      분당      수원      판교    
       a   b   a   b   a   b   a   b   a   b
2017   0   5   1   6   2   7   3   8   4   9
2018  10  15  11  16  12  17  13  18  14  19

In [20]:
# columns 최하위 level > row 하위 level 재배열

my_df.stack()

경기    서울
2017 a 강남   NaN   0.0
       분당   2.0   NaN
       수원   3.0   NaN
       잠실   NaN   1.0
       판교   4.0   NaN
     b 강남   NaN   5.0
       분당   7.0   NaN
       수원   8.0   NaN
       잠실   NaN   6.0
       판교   9.0   NaN
2018 a 강남   NaN  10.0
       분당  12.0   NaN
       수원  13.0   NaN
       잠실   NaN  11.0
       판교  14.0   NaN
     b 강남   NaN  15.0
       분당  17.0   NaN
       수원  18.0   NaN
       잠실   NaN  16.0
       판교  19.0   NaN

### 1.5. `df.swaplevel()`

In [21]:
my_df.swaplevel(1, 0)

서울      경기        
        강남  잠실  분당  수원  판교
a 2017   0   1   2   3   4
b 2017   5   6   7   8   9
a 2018  10  11  12  13  14
b 2018  15  16  17  18  19

In [22]:
my_df.swaplevel(1, 0, axis=1)

강남  잠실  분당  수원  판교
        서울  서울  경기  경기  경기
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

## 2. Sorting 정렬
- `obj.sort_index()` : 인덱스를 기준으로 정렬 (기본값은 ascending=True, 오름차순 정렬)
    - DataFrame, Series
        - axis = 0 : 기본값, 로우 인덱스 기준으로 정렬
        - axis = 1 : 컬럼 인덱스 기준으로 정렬
- `obj.sort_values()` : 값을 기준으로 정렬
    - DataFrame, Series
        - by : 정렬의 기준이 되는 인덱스 값 전달
        - axis = 0 : 기본값, 컬럼을 기준으로 로우 인덱스를 정렬하며 기준값으로 by에 인덱스 컬럼 레벨 또는 컬럼명 전달
        - axis = 1 : 로우 인덱스를 기준으로 컬럼 라벨을 정렬하며 기준값으로 by에 레벨 또는 라벨명 전달

In [23]:
# Series 생성
# 값과 index label이 순서대로 들어가지 않은 Series

sort_sr = pd.Series([2, 3, 1, 7, 0],index=['g', 'a', 'c', 'f', 'd'])

sort_sr

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [24]:
# 인덱스 기준 오름차순 정렬

sort_sr.sort_index()

a    3
c    1
d    0
f    7
g    2
dtype: int64

In [25]:
# 인덱스 기준 내림차순 정렬
# param opt: `ascending=False`
sort_sr.sort_index(ascending=False)

g    2
f    7
d    0
c    1
a    3
dtype: int64

In [26]:
# `obj.sort_index()` > 결과값 return > 원본 영향 X
sort_sr

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [27]:
# values 기준 오름차순 정렬
sort_sr.sort_values()

d    0
c    1
g    2
a    3
f    7
dtype: int64

In [28]:
# values 기준 내림차순 정렬
sort_sr.sort_values(ascending=False)

f    7
a    3
g    2
c    1
d    0
dtype: int64

In [29]:
# 난수 seed 고정
np.random.seed(4)

In [30]:
# DataFrame 생성
# 4 X 5, 난수 정수
# row, columns index > random

rand_df = pd.DataFrame(np.random.randint(20, size=(4, 5)),
                      index=list('hcae'),
                      columns=list('EAFCD'))

rand_df

,E,A,F,C,D
h,14,5,1,8,8
c,18,9,7,13,8
a,4,18,12,6,10
e,3,0,9,6,6


In [31]:
# row index 기준 오름차순 정렬 > `default: axis=0`

rand_df.sort_index()

,E,A,F,C,D
a,4,18,12,6,10
c,18,9,7,13,8
e,3,0,9,6,6
h,14,5,1,8,8


In [32]:
# row index descending > `ascending=False`

rand_df.sort_index(ascending=False)

,E,A,F,C,D
h,14,5,1,8,8
e,3,0,9,6,6
c,18,9,7,13,8
a,4,18,12,6,10


In [33]:
# columns index ascending

rand_df.sort_index(axis=1)

,A,C,D,E,F
h,5,8,8,14,1
c,9,13,8,18,7
a,18,6,10,4,12
e,0,6,6,3,9


In [34]:
# columns index descending

rand_df.sort_index(axis=1, ascending=False)

,F,E,D,C,A
h,1,14,8,8,5
c,7,18,8,13,9
a,12,4,10,6,18
e,9,3,6,6,0


In [35]:
# columns 기준 descending, row 기준 ascending

rand_df.sort_index(axis=1, ascending=False).sort_index()

,F,E,D,C,A
a,12,4,10,6,18
c,7,18,8,13,9
e,9,3,6,6,0
h,1,14,8,8,5


In [36]:
# `obj.sort_index()` > return obj > 원본 영향 X

rand_df

,E,A,F,C,D
h,14,5,1,8,8
c,18,9,7,13,8
a,4,18,12,6,10
e,3,0,9,6,6


In [37]:
# sort by values
# column D > sort by values by ascencding
# `obj.sort_values(axis=0, by="column name")` > default: axis=0 > row arrow
# > 한 col(by="col name")에 대해 sort by values 결과 row에 반영

rand_df.sort_values(by="D")

,E,A,F,C,D
e,3,0,9,6,6
h,14,5,1,8,8
c,18,9,7,13,8
a,4,18,12,6,10


In [38]:
# sort by column A

rand_df.sort_values(by="A")

,E,A,F,C,D
e,3,0,9,6,6
h,14,5,1,8,8
c,18,9,7,13,8
a,4,18,12,6,10


In [39]:
# index label(row) c value sort by ascending
# sorted object > columns
# sort by `row label`

rand_df.sort_values(by='c', axis=1)

,F,D,A,C,E
h,1,8,5,8,14
c,7,8,9,13,18
a,12,10,18,6,4
e,9,6,0,6,3


In [40]:
# sort columns by row 'e' by descending

rand_df.sort_values(by='e', axis=1, ascending=False)

,F,C,D,E,A
h,1,8,8,14,5
c,7,13,8,18,9
a,12,6,10,4,18
e,9,6,6,3,0


In [41]:
# 두 개의 columns에 대해서 정렬 > list로 묶어서 `by="param"`의 param으로 전달
# sort priority: 차례대로 1순위 > 2순위 (1순위 같을 시 > 2순위 비교)
# e.g. sort by values > value가 같을 때 > 다른 index의 value를 기준으로 sort
# 동일한 value가 아니면 > 무조건 1순위 우선

rand_df.sort_values(by=['D', 'E'])

,E,A,F,C,D
e,3,0,9,6,6
h,14,5,1,8,8
c,18,9,7,13,8
a,4,18,12,6,10


In [42]:
# sort e by ascending, sort c by decending > `ascending=[True, False]` 각각 지정

rand_df.sort_values(by=['e', 'c'], axis=1, ascending=[True, False])

,A,E,C,D,F
h,5,14,8,8,1
c,9,18,13,8,7
a,18,4,6,10,12
e,0,3,6,6,9


### > 연습문제

1. 아래의 DataFrame 생성하시오.

- 학생들의 점수는 50 이상 100 미만의 무작위 정수 값을 생성하여 사용

<img src="img/df_sort_practice1.png" width=250 align='left'>

In [43]:
score_df = pd.DataFrame(np.random.randint(50, 100, size=20).reshape(5, 4))

score_df.index = [["Kim", "Park", "Lee", "Jung", "Moon"]]
score_df.index.name = "학생명"

score_df.columns = [[2016, 2016, 2017, 2017],["영어", "수학", "영어", "수학"]]
score_df

2016     2017    
       영어  수학   영어  수학
Kim    52  96   80  58
Park   99  52   73  82
Lee    90  92   95  83
Jung   82  95   78  53
Moon   65  84   81  67

In [44]:
# row index name set
score_df.index.name = "학생명"

score_df

2016     2017    
       영어  수학   영어  수학
Kim    52  96   80  58
Park   99  52   73  82
Lee    90  92   95  83
Jung   82  95   78  53
Moon   65  84   81  67

In [45]:
# columns index name set (by list)
# multi index > `df.columns.set_names(["name1", "name2", ...], inplace=True) or df.columns.names = name1, name2, ...

score_df.columns.set_names(["년도", "과목"], inplace=True)

score_df

년도   2016     2017    
과목     영어  수학   영어  수학
Kim    52  96   80  58
Park   99  52   73  82
Lee    90  92   95  83
Jung   82  95   78  53
Moon   65  84   81  67

<br>

2. 2016년 data만 별도의 DataFrame으로 분리 저장하시오. (deep copy)

In [46]:
df_2016 = score_df[2016].copy()

df_2016

과목,영어,수학
Kim,52,96
Park,99,52
Lee,90,92
Jung,82,95
Moon,65,84


<br>

3. `df_2016`에 대해 학생 이름 기준 sort by ascending

In [47]:
df_2016.sort_index()

과목,영어,수학
Jung,82,95
Kim,52,96
Lee,90,92
Moon,65,84
Park,99,52


<br>

4. `df_2016`에 대해 영어 점수 내림차순, 영어 동점자의 경우 수학 점수 오름차순 정렬.

In [48]:
df_2016["영어"]["Moon"] = 67
df_2016["수학"]["Moon"] = 80

df_2016

과목,영어,수학
Kim,52,96
Park,99,52
Lee,90,92
Jung,82,95
Moon,67,80


In [49]:
df_2016.sort_values(by=["영어", "수학"], ascending=[False, True])

과목,영어,수학
Park,99,52
Lee,90,92
Jung,82,95
Moon,67,80
Kim,52,96


## 3. Data merge 데이터 합병 `pd.merge()`

- pandas 객체의 메서드로 pandas.merge(df1, df2 ...)로 사용

- 두 개의 데이터프레임에 대해 특정 컬럼을 기준으로 합치기

- 주요 파라미터

    - how : 합치는 방식으로 inner(기본값), left, right, outer 방식 존재
        
    - on : 합치는 기준으로 두 개의 데이터프레임에 공통으로 존재하는 컬럼명을 사용해야함 (기본값=None)

In [50]:
# 고객번호와 고객명을 저장할고 있는 데이터프레임 생성
# 딕셔너리 타입으로 데이터프레임 데이터 생성 : key:col, value-> list():row

df_name = pd.DataFrame({'no':[30, 31, 32, 33, 34],
            'name':['김파이썬', '이장고', '박팬더스', '강넘파이', '최주피터']})

df_name

,no,name
0,30,김파이썬
1,31,이장고
2,32,박팬더스
3,33,강넘파이
4,34,최주피터


In [51]:
# 고객 번호와 주문수량을 저장하고 있는 데이터프레임 생성
# 다른 고객번호(no)를 일부 지정
df_amt = pd.DataFrame({'no':[30, 32, 33, 40, 41],
                   'amount':[100, 40, 130, 40, 60]})

df_amt

,no,amount
0,30,100
1,32,40
2,33,130
3,40,40
4,41,60


In [52]:
# 가장 기본적인 merge 방식 : common columns 기준으로, 두 DataFrame의 집합에 모두 존재하는 (교집합)item만 추출
# 기본 동작: `how='inner'`

# common columns between `df_name` and `df_amt` : `no`
# common columns 내부의 common values : 30, 32, 33

pd.merge(df_name, df_amt)

,no,name,amount
0,30,김파이썬,100
1,32,박팬더스,40
2,33,강넘파이,130


### 3.1. `how=''`

In [53]:
# `how='outer'` > 결합 기준으로 common comlumns 사용
# 교집합이 아닌 부분 > NaN으로 처리 (누락 X)
# 합집합 형태로 만듦

pd.merge(df_name, df_amt, how='outer')

,no,name,amount
0,30,김파이썬,100.0
1,31,이장고,NaN
2,32,박팬더스,40.0
3,33,강넘파이,130.0
4,34,최주피터,NaN
5,40,NaN,40.0
6,41,NaN,60.0


<br>

- `how='left'`: 첫 번째로 전달한 df의 data와 교집합 data 결합

In [54]:
pd.merge(df_name, df_amt, how='left')

,no,name,amount
0,30,김파이썬,100.0
1,31,이장고,NaN
2,32,박팬더스,40.0
3,33,강넘파이,130.0
4,34,최주피터,NaN


<br>

- `how='right'`: 두 번째로 전달한 df의 data와 교집합 data 결합

In [55]:
pd.merge(df_name, df_amt, how='right')

,no,name,amount
0,30,김파이썬,100
1,32,박팬더스,40
2,33,강넘파이,130
3,40,NaN,40
4,41,NaN,60


- 공통 컬럼이 두 개 이상인 경우

In [56]:
# 고객명, 날짜, 폰 정보를 저장하고 있는 데이터프레임 생성
df_phone = pd.DataFrame({'고객명':['김파이썬', '이장고', '박팬더스'],
                   '날짜':['2022-10-22', '2022-10-23', '2022-10-24'],
                   '정보':['010', '011', '019']})
df_phone

,고객명,날짜,정보
0,김파이썬,2022-10-22,010
1,이장고,2022-10-23,011
2,박팬더스,2022-10-24,019


In [57]:
# 고객명, 성별정보를 저장하고 있는 데이터프레임 생성
df_sex = pd.DataFrame({'고객명':['김파이썬', '박팬더스', '최넘파이'],
                   '정보':['F', 'M', 'M']})
df_sex

,고객명,정보
0,김파이썬,F
1,박팬더스,M
2,최넘파이,M


In [58]:
# common columns: 2개 이상 > no less parameter than result

pd.merge(df_phone, df_sex)

,고객명,날짜,정보


### 3.2. `on`, `left_on`, `right_on`

<br>

- `on` option parameter: 공통된 columns가 여럿인 경우 결합 기준 column을 지정

In [59]:
# merge key column: '고객명'
# common column result: '고객명', '정보' > '정보' column 결과 확인
# default: `how='inner'`

pd.merge(df_phone, df_sex, on='고객명')

,고객명,날짜,정보_x,정보_y
0,김파이썬,2022-10-22,010,F
1,박팬더스,2022-10-24,019,M


<br>

- `left_on`, `right_on` : 두 개의 데이터프레임에 대해서 서로 다른 기준컬럼을 지정
- e.g. 동일한 속성의 자료를 저장하는 컬럼인데 표기하는 이름이 다른 경우

In [60]:
# 고객이름, 날짜, 구매금액을 저장하고 있는 데이터프레임 생성
df_date = pd.DataFrame({'고객이름':['김파이썬', '박팬더스', '강주피터'],
                   '날짜':['2020-01-01', '2020-02-01', '2020-02-15'],
                   '구매금액':[1, 2, 3]})
df_date

,고객이름,날짜,구매금액
0,김파이썬,2020-01-01,1
1,박팬더스,2020-02-01,2
2,강주피터,2020-02-15,3


In [61]:
# 고객명, 성별을 저장하고 있는 데이터프레임을 생성
df_sex = pd.DataFrame({'고객명':['김파이썬', '박팬더스'],
                   '성별':['F', 'M']})
df_sex

,고객명,성별
0,김파이썬,F
1,박팬더스,M


In [62]:
# pd.merge(df_date, df_sex) # MergeError: No common columns

In [63]:
# left(df_date), right(df_sex)에서 common columns 각각 지정

df_tmp = pd.merge(df_date, df_sex, left_on='고객이름', right_on='고객명')

df_tmp

,고객이름,날짜,구매금액,고객명,성별
0,김파이썬,2020-01-01,1,김파이썬,F
1,박팬더스,2020-02-01,2,박팬더스,M


In [64]:
# `.drop()` method > column '고객명' 삭제

df_tmp.drop('고객명', axis=1, inplace=True)

df_tmp

,고객이름,날짜,구매금액,성별
0,김파이썬,2020-01-01,1,F
1,박팬더스,2020-02-01,2,M


## 4. Data Concatenate 데이터 연결 `pd.concat()`
- 특정 key를 기준으로 데이터를 합치는 것이 아니라 행, 열 기준으로 데이터를 연결 > data crawling에서 많이 사용됨
- 주요 파라미터
    - axis : 0 / 행 방향(기본값)이며 컬럼을 key로 합치고, 1 / 열 방향으로 로우를 key로 합침
    - join : 데이터프레임끼리 연결할 때 합치는 방법으로 outer(기본값), inner 방식 존재
    - **ignore_index** : 합친 후 기존 인덱스를 유지 또는 새로운 인덱스를 지정

In [65]:
# Two Series have only two common index label: 'a', 'c'
sr_1 = pd.Series([1, 2, 3], index=list('abc'))
sr_2 = pd.Series([5, 6, 7, 8], index=list('abfh'))

print(sr_1)
print(sr_2)

a    1
b    2
c    3
dtype: int64
a    5
b    6
f    7
h    8
dtype: int64


<br>

- connect(`concat`) between two Series
- default: `axis=0` > row arrow
- 첫 번째로 전달된 obj가 위에, 두 번째로 전달된 obj가 아래로 추가(연결)
- index label은 기존 값 유지

In [66]:
pd.concat([sr_1, sr_2])

a    1
b    2
c    3
a    5
b    6
f    7
h    8
dtype: int64

In [67]:
# 새로운 index로 초기화 > `ignore_index=True`

pd.concat([sr_1, sr_2], ignore_index=True)

0    1
1    2
2    3
3    5
4    6
5    7
6    8
dtype: int64

In [68]:
# columns arrow concat: `axis=1`, 두 개의 Series 연결 > 하나의 DataFrame
# columns 길이(row 개수)가 다른 경우

pd.concat([sr_1, sr_2], axis=1)

,0,1
a,1.0,5.0
b,2.0,6.0
c,3.0,NaN
f,NaN,7.0
h,NaN,8.0


In [69]:
# 열 이름 설정하며 연결하기: `keys` optinal parameter에 columns name을 list로 전달

pd.concat([sr_1, sr_2], axis=1, keys=['c1', 'c2'], sort=False)

,c1,c2
a,1.0,5.0
b,2.0,6.0
c,3.0,NaN
f,NaN,7.0
h,NaN,8.0


### > 연습문제 `pd.concat()`

In [70]:
# 고객명, 날짜, 구매금액
df_date = pd.DataFrame({'고객명':['김파이썬', '이장고', '박팬더스'],
                   '날짜':['2022-10-22', '2022-10-23', '2022-12-14'],
                   '구매금액':[1, 2, 3]})
df_date

,고객명,날짜,구매금액
0,김파이썬,2022-10-22,1
1,이장고,2022-10-23,2
2,박팬더스,2022-12-14,3


In [71]:
# 고객명, 성별
df_sex = pd.DataFrame({'고객명':['김파이썬', '최넘파이'],
                   '성별':['F', 'M']})
df_sex

,고객명,성별
0,김파이썬,F
1,최넘파이,M


In [72]:
# row arrow DataFrame concat > row 개수 up

pd.concat([df_date, df_sex], axis=0)

,고객명,날짜,구매금액,성별
0,김파이썬,2022-10-22,1.0,NaN
1,이장고,2022-10-23,2.0,NaN
2,박팬더스,2022-12-14,3.0,NaN
0,김파이썬,NaN,NaN,F
1,최넘파이,NaN,NaN,M


In [73]:
# columns arrow DataFrame concat

pd.concat([df_date, df_sex], axis=1)

,고객명,날짜,구매금액,고객명,성별
0,김파이썬,2022-10-22,1,김파이썬,F
1,이장고,2022-10-23,2,최넘파이,M
2,박팬더스,2022-12-14,3,NaN,NaN


## 5. 집계
1. `df.groupby(column name)`
    - 특정 속성을 기준으로 묶어서 다양한 집계 함수 적용
    - 대표적인 집계 함수
        - `.sum()` : 총합
        - `.mean()` : 평균값
        - `.min()` : 최소값
        - `.max()` : 최대값
        - `.count()` : 개수
        - `.std()` : 표준편차
2. pivot table 
    - `df.pivot('col name' to be row, 'col name' to be column, 'col name' to be tuple values, 집계함수)`
    - 일차원으로 columns, row가 단순 나열된 형식 > 데이터를 파악하는데 부적합 > pivot을 통해 계층 색인 및 형태 변경을 수행

In [74]:
# 엑셀 데이터 적재
# if Os Error > engine='python'
# if Unicode, Encoding Error > encoding='utf-8' or 'cp949' or 'utf-16'

pop_data = pd.read_excel('data/인구수예제.xlsx')
pop_data

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190
5,서울,서대문구,2013,95,111,206
6,서울,서대문구,2014,149,150,299
7,서울,서대문구,2015,106,77,183
8,서울,서대문구,2016,56,109,165
9,서울,서대문구,2017,82,96,178


In [75]:
pop_data.shape

(50, 6)

In [76]:
# 상위 5개 data만 조회 > `df.head()`: index 기준

pop_data.head()

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190


In [77]:
# 하위 5개 data 조회

pop_data.tail()

,도시,자치구,연도,남자인구,여자인구,총인구
45,부산,동래구,2013,83,65,148
46,부산,동래구,2014,139,87,226
47,부산,동래구,2015,147,115,262
48,부산,동래구,2016,61,102,163
49,부산,동래구,2017,132,105,237


In [78]:
# 상위 N개 조회: `df.head(N)`

pop_data.head(10)

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190
5,서울,서대문구,2013,95,111,206
6,서울,서대문구,2014,149,150,299
7,서울,서대문구,2015,106,77,183
8,서울,서대문구,2016,56,109,165
9,서울,서대문구,2017,82,96,178


### 5.1. `df.groupby(column name)`

In [79]:
# column index: '자치구'별 '남자인구', '여자인구' 각각의 총합
# > `df.groupby('자치구')` + `[['남자인구', '여자인구']]` + `.sum()`
# '연도' = 2013 ~ 2017

pop_data.groupby('자치구')[['남자인구', '여자인구']].sum()

,남자인구,여자인구
자치구,,
강남구,539,513
도봉구,485,550
동래구,562,474
동작구,454,582
서대문구,488,543
송파구,415,559
수영구,502,559
영등포구,629,562
종로구,483,373


In [80]:
# 도시별 남여 인구 각각의 총합

pop_data.groupby('도시')[['남자인구', '여자인구']].sum()

,남자인구,여자인구
도시,,
부산,1684,1548
서울,3493,3682


In [81]:
# 집계 조건 2개 >'연도'별, '도시'별 
# column name 지정 X > 집계 기준 column 제외 나머지 다 집계

pop_data.groupby(['연도', '도시']).sum()

남자인구  여자인구   총인구
연도   도시                  
2013 부산   341   262   603
     서울   758   769  1527
2014 부산   314   369   683
     서울   882   691  1573
2015 부산   331   266   597
     서울   541   710  1251
2016 부산   276   376   652
     서울   671   722  1393
2017 부산   422   275   697
     서울   641   790  1431

In [82]:
# '도시'별, '연도'별 > '총인구' 평균

pop_data.groupby(['도시', '연도'])[['총인구']].mean()

총인구
도시 연도              
부산 2013  201.000000
   2014  227.666667
   2015  199.000000
   2016  217.333333
   2017  232.333333
서울 2013  218.142857
   2014  224.714286
   2015  178.714286
   2016  199.000000
   2017  204.428571

In [83]:
# data analysis에 proper shape로 변경

pop_data.groupby(['도시', '연도'])[['총인구']].mean().unstack(0)

총인구            
도시            부산          서울
연도                          
2013  201.000000  218.142857
2014  227.666667  224.714286
2015  199.000000  178.714286
2016  217.333333  199.000000
2017  232.333333  204.428571

## 6.통계
- 주요 통계 함수
    - value_count() : 각 고유값의 중복개수
    - count : NaN 값을 제외한 값의 개수
    - describe : 각 컬럼에 대한 요약통계 계산(count, mean, std, min, 1사분위수, 중위값, 3사분위수, max)
    - min, max : 최소, 최대 값
    - sum : 총 합
    - cumsum : 누적합
    - mean : 평균
    - median : 중위값(전체 데이터를 나열 했을 때 중간에 위치한 값)
    - var : 분산(데이터가 전체적으로 흩어진 정도, 편차제곱의 평균)
    - std : 표준편차 = 분산의 양의 제곱근
- 주요 파라미터
    - axis : 연산의 기준이 되는 축, axis=0(기본값)이면 행 방향으로 axis=1이면 열 방향으로 적용
    - skipna : NaN 값을 제외할지 여부를 설정, 기본값 = True

In [94]:
np.random.seed(25)

In [95]:
# 샘플 데이터 생성
my_index = ['Kim', 'Park', 'Lee', 'Jung', 'Moon']
my_columns = [[2016, 2016, 2017, 2017], ['영어', '수학', '영어', '수학']]

df = pd.DataFrame(np.random.randint(50, 100, (5, 4)),
                  index=my_index, columns=my_columns)

df.index.set_names('학생명', inplace=True)

df.columns.set_names(['연도', '과목'], inplace=True)

df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    54  76   65  73
Park   94  58   78  54
Lee    75  81   55  51
Jung   89  53   74  53
Moon   70  95   53  51

In [96]:
# 기술통계, 요약통계

df.describe()

연도          2016                  2017           
과목            영어         수학         영어         수학
count   5.000000   5.000000   5.000000   5.000000
mean   76.400000  72.600000  65.000000  56.400000
std    15.915401  17.184295  11.113055   9.370165
min    54.000000  53.000000  53.000000  51.000000
25%    70.000000  58.000000  55.000000  51.000000
50%    75.000000  76.000000  65.000000  53.000000
75%    89.000000  81.000000  74.000000  54.000000
max    94.000000  95.000000  78.000000  73.000000

In [97]:
# 2017 성적만 분리해서 deep copy

df_2017 = df[2017].copy()

df_2017

과목,영어,수학
학생명,,
Kim,65,73
Park,78,54
Lee,55,51
Jung,74,53
Moon,53,51


In [98]:
df_2017.shape

(5, 2)

In [99]:
# 1. count() - NaN을 제외한 data의 개수 (row 개수)
# 축 기준 설정: axis=0(column축, 행 개수) or axis=1(row축, 열 개수)
# row 개수: axis=0

df_2017.count()

과목
영어    5
수학    5
dtype: int64

In [100]:
# 열 개수 > axis=1

df_2017.count(axis=1)

학생명
Kim     2
Park    2
Lee     2
Jung    2
Moon    2
dtype: int64

In [101]:
# 엑셀 데이터 적재
# Os error -> engine='python'
# Unicode, Encoding -> encoding='utf-8' or 'cp949'
data = pd.read_excel('data/인구수예제.xlsx')
data

# 통계분석 > 문자통계분석, 숫자통계분석

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190
5,서울,서대문구,2013,95,111,206
6,서울,서대문구,2014,149,150,299
7,서울,서대문구,2015,106,77,183
8,서울,서대문구,2016,56,109,165
9,서울,서대문구,2017,82,96,178


In [102]:
# value_counts(): 범주형 데이터(문자열 등)에 대한 각 분류값의 개수
# '도시' data 사용

data['도시'].value_counts()

서울    35
부산    15
Name: 도시, dtype: int64

In [104]:
# 2. sum(): 해당 columns이나 row의 총합을 계산
# 기본 축 방향: axis=0 > column별 총합

df_2017.sum()

과목
영어    325
수학    282
dtype: int64

In [105]:
# 학생 별 점수 총합
# axis=1 > row별 더하기

df_2017.sum(axis=1)

학생명
Kim     138
Park    132
Lee     106
Jung    127
Moon    104
dtype: int64

In [106]:
# 3. mean()
# default: axis=0
# 과목별 평균

df_2017.mean()

과목
영어    65.0
수학    56.4
dtype: float64

In [107]:
# 학생별 평균: axis=1

df_2017.mean(axis=1)

학생명
Kim     69.0
Park    66.0
Lee     53.0
Jung    63.5
Moon    52.0
dtype: float64

### > 연습문제

In [108]:
df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    54  76   65  73
Park   94  58   78  54
Lee    75  81   55  51
Jung   89  53   74  53
Moon   70  95   53  51

In [132]:
# 1. 2016, 2017 년도별, 과목별 평균 성적 출력

df_sum = df[[2016, 2017]].mean()

df_sum

연도    과목
2016  영어    76.4
      수학    72.6
2017  영어    65.0
      수학    56.4
dtype: float64

In [133]:
# 2. 1번의 년도별, 과목별 평균의 row=연도, columns=과목이 되도록 수정.

df_sum.unstack()

과목,수학,영어
연도,,
2016,72.6,76.4
2017,56.4,65.0


In [134]:
# 3. 학생별 전체평균 출력

df.mean(axis=1)

학생명
Kim     67.00
Park    71.00
Lee     65.50
Jung    67.25
Moon    67.25
dtype: float64